In [3]:
## INSPIRED FROM THE https://github.com/cmasch/word-embeddings-from-scratch/blob/master/Create_Embeddings.ipynb


!pip install gensim 
!pip install pandas
import gensim, logging, os, re, string, tensorflow
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from gensim.utils import simple_preprocess
import pandas as pd 
import numpy as np 

print('gensim version: \t%s'     % gensim.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)

    100% |████████████████████████████████| 23.6MB 1.7MB/s 
    100% |████████████████████████████████| 1.4MB 16.7MB/s 
    100% |████████████████████████████████| 133kB 27.6MB/s 
    100% |████████████████████████████████| 61kB 23.8MB/s 
    100% |████████████████████████████████| 5.1MB 7.0MB/s 
    100% |████████████████████████████████| 552kB 26.2MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
gensim version: 	3.6.0
TensorFlow version: 	1.12.0


In [10]:
def prepare_movie_dataset(train_start,train_end,test_start,test_end) : 

    dataset = pd.read_csv('movie_reviews.csv')
    print('dataset size : ',dataset.shape[0])

    train_data = dataset[train_start:train_end]
    test_data = dataset[test_start:test_end]
    
    print('Train_X : ',train_data.shape[0])
    print('Test_X  : ',test_data.shape[0])

    test_reviews = np.array(test_data['review'])
    test_sentiments = np.array(test_data['sentiment'])
    train_reviews=np.array(train_data['review'])
    return train_reviews,test_reviews,test_sentiments

train_x,test_x,test_y=prepare_movie_dataset(0,50000,1,2)

dataset size :  50000
Train_X :  50000
Test_X  :  1


In [11]:

# For displaying gensim logs
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)


EMBEDDING_SIZE = 300


model = gensim.models.Word2Vec(train_x, size=EMBEDDING_SIZE)




INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO : PROGRESS: at sentence #10000, processed 13091245 words, keeping 164 word types
INFO : PROGRESS: at sentence #20000, processed 26189472 words, keeping 175 word types
INFO : PROGRESS: at sentence #30000, processed 39271702 words, keeping 180 word types
INFO : PROGRESS: at sentence #40000, processed 52320923 words, keeping 192 word types
INFO : collected 206 word types from a corpus of 65471551 raw words and 50000 sentences
INFO : Loading a fresh vocabulary
INFO : effective_min_count=5 retains 160 unique words (77% of original 206, drops 46)
INFO : effective_min_count=5 leaves 65471467 word corpus (99% of original 65471551, drops 84)
INFO : deleting the raw counts dictionary of 206 items
INFO : sample=0.001 downsamples 31 most-common words
INFO : downsampling leaves estimated 14234449 word corpus (21.7% of prior 65471467)
INFO : estimated required memory for 160 wor

TypeError: ignored

In [13]:
weights     = model.wv.vectors
index_words = model.wv.index2word

vocab_size    = weights.shape[0]
embedding_dim = weights.shape[1]

print('Shape of weights:', weights.shape)
print('Vocabulary size: %i' % vocab_size)
print('Embedding size: %i'  % embedding_dim)

## WRITE THE METADATA TO .TSV FILES

MODEL_DIR='.'
with open(os.path.join('metadata.tsv'), 'w') as f:
    f.writelines("\n".join(index_words))

# Required if you re-run without restarting the kernel
tf.reset_default_graph()
    
W = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]), trainable=False, name="W")
embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])

embedding_init = W.assign(embedding_placeholder)
writer = tf.summary.FileWriter(MODEL_DIR, graph=tf.get_default_graph())
saver = tf.train.Saver()

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = W.name
embedding.metadata_path = './metadata.tsv'
projector.visualize_embeddings(writer, config)

with tf.Session() as sess:
    sess.run(embedding_init, feed_dict={embedding_placeholder: weights})
    save_path = saver.save(sess, os.path.join(MODEL_DIR, "model.cpkt"))

Shape of weights: (160, 300)
Vocabulary size: 160
Embedding size: 300


In [0]:
os.listdir('.')

['.config',
 '.ipynb_checkpoints',
 'movie_reviews.csv',
 'emb_yelp',
 'sample_data']

In [0]:
#model.wv.most_similar(positive=['coffee'], topn=10)


In [0]:
## CONVERT THE BINARY TO THE TEXT FILE

from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('word2vec.bin', binary=True)
model.save_word2vec_format('word2vec.txt', binary=False)

INFO : loading projection weights from word2vec.bin


UnicodeDecodeError: ignored